Creating a Unigram Tokenizer on the new sentences (without considering the definitions)
----------------------------------
We extracted new sentences from the book <i style="color: cyan">Grammaire de Wolof Moderne</i> by Pathé Diagne. Since we want to test the relevancy of the new sentences to our translation task, let us create a tokenizer for them. It is done in order to train the T5 model on it and see if we obtain a better performance that with the original corpora. 

The process is almost the same as in [processing_4](text_processing4.ipynb) excepted that we will create another custom dataset for the custom transformer model.

Let us import the necessary libraries.

In [90]:
# for creating the tokenizer
from tokenizers import (
    decoders,
    models,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
    normalizers
)

# for importing and manipulating the sentences
import pandas as pd
import random

# for plotting the box plot of the sequence lengths
import plotly.express as px

# for loading sentences with the custom dataset
from torch.utils.data import DataLoader

#### Load dataset and create generator

We will create one tokenizer for both of the French and Wolof corpora because the `T5` model' understand only one embedding layer. So we must create one generator for both of the French and Wolof corpora. 

In [91]:
# load sentences
sentences = pd.read_csv("data/extractions/new_data/sentences.csv")

# initialize a batch size
BATCH_SIZE = 400

# create generators (for the corpora)
def generate_sents():
    
    # recuperate the sentences
    french_sents = sentences['french'].to_list() 
    
    wolof_sents = sentences['wolof'].to_list() 
    
    sents = french_sents + wolof_sents
    
    for i in range(1, len(sents), BATCH_SIZE):
        
        yield sents[i:i+BATCH_SIZE]

#### Initialize the tokenizer

In [92]:
tokenizer = Tokenizer(models.Unigram())

#### Add normalizer

In [93]:
tokenizer.normalizer = normalizers.Replace(" {2,}", " ")

#### Configure the pre-tokenizers

We will use the Metaspace pre-tokenizer which separates the words considering the spaces between them. It will replace the space by a character (by default the underscore "_").

In [94]:
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

#### Initialize the trainers

We will provide all of the necessary special tokens to the Trainer. 

**Notice that a sentence can be a group of words separated by ending marks and not only one group of words. Then we can, for example, tokenize the following sentences**: `<sep>sentence1.sentence2.sentence3<cls>` **or** `<sep>sentence1.<sep>sentence2.<cls>`. **But, the second sentence is composed of two separate groups. Then the two sentences will have different type ids.** 

In [95]:
special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]

In [96]:
trainer = trainers.UnigramTrainer(special_tokens=special_tokens, unk_token = "<unk>") # let us take the default vocab size

#### Train the tokenizer

In [97]:
tokenizer.train_from_iterator(generate_sents(), trainer)

Let us print the vocab size.

In [98]:
print(f"Number of tokens: {tokenizer.get_vocab_size()}")

Number of tokens: 1937


#### Initialize the post-processor

We can not need the TemplateProcessor to train our corpora in a Sequence To Sequence model, but we will add it to our tokenizer. We can use it for another type of model. 

In [99]:
# let us recuperate the sep and cls ids
cls_token_id = tokenizer.token_to_id("<cls>")

sep_token_id = tokenizer.token_to_id("<sep>")

print(cls_token_id, sep_token_id)

0 1


In [100]:
# Initialize the post processor
tokenizer.post_process = processors.TemplateProcessing(
    single="$A:0 <sep>:0 <cls>:2",
    pair="$A:0 <sep>:0 $B:1 <sep>:1 <cls>:2",
    special_tokens=[("<sep>", sep_token_id), ("<cls>", cls_token_id)]
)

#### Initialize the decoder

In [101]:
tokenizer.decoder = decoders.Metaspace()

#### Save the tokenizer

In [102]:
tokenizer.save("wolof-translate/wolof_translate/tokenizers/t5_tokenizers/tokenizer_v3.json")

#### Make a little example

Let us recuperate random sentences from the corpora and tokenize them.

In [103]:
random.seed(200)

french_sentence = random.choice(sentences['french']) + "</s>"

wolof_sentence = random.choice(sentences['wolof']) + "</s>"

In [104]:
# print the french sentence
french_sentence

'Fais sortir tout cheval que tu vois !</s>'

In [105]:
# print the wolof sentence
wolof_sentence

'Yeena doon dem</s>'

In [106]:
french_encoding = tokenizer.encode(french_sentence)

print("French tokens")
print(french_encoding.tokens)

print("French ids")
print(french_encoding.ids)

French tokens
['▁Fa', 'is', '▁sortir', '▁tout', '▁cheval', '▁que', '▁tu', '▁vois', '▁!', '</s>']
French ids
[164, 251, 537, 134, 394, 27, 44, 220, 18, 6]


In [107]:
wolof_encoding = tokenizer.encode(wolof_sentence)

print("Wolof tokens")
print(wolof_encoding.tokens)

print("Wolof ids")
print(wolof_encoding.ids)

Wolof tokens
['▁Yee', 'na', '▁doon', '▁dem', '</s>']
Wolof ids
[860, 210, 205, 16, 6]


#### Creating the T5 custom dataset for the new sentences

We have two possibilities to use the tokenizer for fine-tuning a T5 model. 

- We can use the `PreTrainedTokenizerFast` class for which we will provide the different special tokens.

In [108]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer1 = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
    padding_side="left",
)

- Or give directly the tokenizer to the `T5TokenizerFast` class.

In [109]:
from transformers import XLNetTokenizerFast

In [110]:
from transformers import T5TokenizerFast

wrapped_tokenizer2 = T5TokenizerFast(
    tokenizer_object=tokenizer
)

Let us give them the sentences that we use as example. 

In [111]:
fr_encoding = wrapped_tokenizer1(french_sentence, max_length=15, padding='max_length', truncation=True)

fr_encoding

{'input_ids': [3, 3, 3, 3, 3, 164, 251, 537, 134, 394, 27, 44, 220, 18, 6], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [112]:
wf_encoding = wrapped_tokenizer2(wolof_sentence, max_length=15, padding='max_length', truncation=True)

wf_encoding

{'input_ids': [860, 210, 205, 16, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], 'attention_mask': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

Let us decode the wolof sentence.

In [113]:
wrapped_tokenizer1.decode(wf_encoding.input_ids, skip_special_tokens=True)

'Yeena doon dem'

We can see that the `T5Tokenizer` add padding to the right side of the sequence while the `PretrainedTokenizer` add the padding to the left side. We can change the padding side from the settings. But, for the next steps, let us directly use the `T5Tokenizer`.

**Note that we can augment the sentences when generating them like we did when using the `GPT2 model`.** See the following notebook, [augmentation](text_augmentation.ipynb), for discussion on the augmentation method that we will use. And for a more clear explanation of the augmentation methods in NLP tasks and training, look at the following article from the web [augment_or_not](https://direct.mit.edu/coli/article/48/1/5/108844/To-Augment-or-Not-to-Augment-A-Comparative-Study).

Let us verify, before creating the custom dataset, the max length that we can get from the corpora' tokens without considering the augmentation. We must for that trace the box plot of the lengths and identify the range in which we will sample the max length of the sequences.

In [114]:
length = []

for sent in sentences['french'].to_list() + sentences['wolof'].to_list():
    
    len_ids = len(wrapped_tokenizer2(sent).input_ids)
    
    length.append(len_ids)

        

In [115]:
fig = px.box(x = length)

fig.update_layout({'xaxis': {'title': 'length'}})

The upper fence is of **15** and the max length is equal to **43**. Then we will test any value between the two. 

But considering the augmentation we can obtain more than the value that we will take because it will add modifications on the words and then it can recognize only parts of them and divide them in multiple other tokens. We will add to the max length the fifth of it. 